In [1]:
import glob
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fbprophet import Prophet
from sklearn import neighbors, ensemble, tree, metrics
from statsmodels.graphics import tsaplots
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import *

%matplotlib notebook

package_dir = os.getcwd()

print(package_dir)

/home/sebastian/Programming/Bachelorthesis


# Version 1

In [2]:
df = pd.DataFrame()

for file in glob.glob('res/*2013_timeseries.csv'):
    # read single file, index on StationEoI and DatetimeEnd
    read = pd.read_csv(file,
                 encoding="utf-16", parse_dates=[13, 14],
                 infer_datetime_format=True,
                 index_col=[14])
    # drop 'bulk' files because they have different averaging
    bulks = read.SamplingPoint.str.lower().str.contains('bulk')
    clean = read[~bulks].copy()
    
    # ignore unnecessary columns
    clean.drop(columns=['Countrycode', 'Namespace', 'AirQualityNetwork',
                 'AirQualityStation', 'SamplingPoint', 'Sample',
                 'SamplingProcess', 'AirPollutantCode',
                 'DatetimeBegin', 'Validity', 'Verification',
                 'AveragingTime'],
        inplace=True)
    
    
    df = pd.concat([df, clean])

# make pollutant a column for better memory usage
#df = df.pivot(columns='AirPollutant')

# make names shorter    
df.index.names = ['Timestamp']
#df.columns.names = [None, 'Pollutant']

df = df.sort_index()
#df = df.groupby(level=[0]).first()

df = df.interpolate()

In [3]:
df.loc['2013-01-01 00:00']

,AirQualityStationEoICode,AirPollutant,Concentration,UnitOfMeasurement
Timestamp,,,,
2013-01-01,DESN077,NO,66.697,µg/m3
2013-01-01,DESN025,C6H5-CH3,1.505,µg/m3
2013-01-01,DESN077,NOX as NO2,145.155,µg/m3
2013-01-01,DESN059,PM10,110.953,µg/m3
2013-01-01,DESN082,O3,37.811,µg/m3
2013-01-01,DESN077,PM10,1218.325,µg/m3
2013-01-01,DESN059,NOX as NO2,26.546,µg/m3
2013-01-01,DESN059,NO,4.244,µg/m3
2013-01-01,DESN025,PM10,200.165,µg/m3


In [4]:
df.columns

Index(['AirQualityStationEoICode', 'AirPollutant', 'Concentration',
       'UnitOfMeasurement'],
      dtype='object')

In [5]:
df[df.AirQualityStationEoICode == 'DESN059']

,AirQualityStationEoICode,AirPollutant,Concentration,UnitOfMeasurement
Timestamp,,,,
2013-01-01 00:00:00,DESN059,PM10,110.953,µg/m3
2013-01-01 00:00:00,DESN059,NOX as NO2,26.546,µg/m3
2013-01-01 00:00:00,DESN059,NO,4.244,µg/m3
2013-01-01 00:00:00,DESN059,O3,48.246,µg/m3
2013-01-01 01:00:00,DESN059,NO,1.000,µg/m3
2013-01-01 01:00:00,DESN059,O3,56.526,µg/m3
2013-01-01 01:00:00,DESN059,PM10,119.628,µg/m3
2013-01-01 01:00:00,DESN059,NOX as NO2,9.654,µg/m3
2013-01-01 02:00:00,DESN059,PM10,15.053,µg/m3


# Version 2

In [2]:
df = pd.DataFrame()

for file in glob.glob('res/*2013_timeseries.csv'):
    # read single file, index on StationEoI and DatetimeEnd
    read = pd.read_csv(file,
                 encoding="utf-16", parse_dates=[13, 14],
                 infer_datetime_format=True,
                 index_col=[14])
    # drop 'bulk' files because they have different averaging
    bulks = read.SamplingPoint.str.lower().str.contains('bulk')
    clean = read[~bulks].copy()
    
    # ignore unnecessary columns
    clean.drop(columns=['Countrycode', 'Namespace', 'AirQualityNetwork',
                 'AirQualityStation', 'SamplingPoint', 'Sample',
                 'SamplingProcess', 'AirPollutantCode',
                 'DatetimeBegin', 'Validity', 'Verification',
                 'AveragingTime'],
        inplace=True)
    
    
    df = pd.concat([df, clean])

# make pollutant a column for better memory usage
df = df.pivot_table(columns='AirPollutant',
                   index=[df.index, 'AirQualityStationEoICode', 'UnitOfMeasurement'],
                   values='Concentration').reset_index(level=[1,2])

# make names shorter    
df.index.names = ['Timestamp']
#df.columns.names = [None, 'Pollutant']

df = df.sort_index()
#df = df.groupby(level=[0]).first()



In [7]:
df

AirPollutant,AirQualityStationEoICode,UnitOfMeasurement,As in PM10,BaP in PM10,Benzo(a)anthracene in PM10,Benzo(b)fluoranthene in PM10,Benzo(j)fluoranthene in PM10,Benzo(k)fluoranthene in PM10,C6H5-CH3,C6H6,...,"Indeno-(1,2,3-cd)pyrene in PM",NO,NOX as NO2,Ni in PM10,O3,OC in PM10,PM10,PM2.5,Pb in PM10,SO2
Timestamp,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,DESN025,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,1.505,1.959,...,NaN,36.848,102.757,NaN,NaN,NaN,200.1650,NaN,NaN,27.458
2013-01-01 00:00:00,DESN059,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.244,26.546,NaN,48.246,NaN,110.9530,NaN,NaN,NaN
2013-01-01 00:00:00,DESN077,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,66.697,145.155,NaN,NaN,NaN,1218.3250,NaN,NaN,NaN
2013-01-01 00:00:00,DESN082,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,37.811,NaN,NaN,NaN,NaN,NaN
2013-01-01 01:00:00,DESN025,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,2.820,2.279,...,NaN,42.785,110.759,NaN,NaN,NaN,205.2370,NaN,NaN,10.514
2013-01-01 01:00:00,DESN059,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000,9.654,NaN,56.526,NaN,119.6280,NaN,NaN,NaN
2013-01-01 01:00:00,DESN077,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.570,19.558,NaN,NaN,NaN,235.7340,NaN,NaN,NaN
2013-01-01 01:00:00,DESN082,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,50.101,NaN,NaN,NaN,NaN,NaN
2013-01-01 02:00:00,DESN025,µg/m3,NaN,NaN,NaN,NaN,NaN,NaN,1.262,1.322,...,NaN,13.197,39.768,NaN,NaN,NaN,61.5880,NaN,NaN,5.561


In [17]:
df.columns.drop('UnitOfMeasurement')

Index(['AirQualityStationEoICode', 'As in PM10', 'BaP in PM10',
       'Benzo(a)anthracene in PM10', 'Benzo(b)fluoranthene in PM10',
       'Benzo(j)fluoranthene in PM10', 'Benzo(k)fluoranthene in PM10',
       'C6H5-CH3', 'C6H6', 'Cd in PM10', 'Dibenzo(ah)anthracene in PM10',
       'EC in PM10', 'Indeno-(1,2,3-cd)pyrene in PM', 'NO', 'NOX as NO2',
       'Ni in PM10', 'O3', 'OC in PM10', 'PM10', 'PM2.5', 'Pb in PM10', 'SO2'],
      dtype='object', name='AirPollutant')

In [23]:
df[df.AirQualityStationEoICode == 'DESN059'].dropna(1, how='all').drop(columns=['UnitOfMeasurement', 'AirQualityStationEoICode'])

AirPollutant,EC in PM10,NO,NOX as NO2,O3,OC in PM10,PM10,PM2.5
Timestamp,,,,,,,
2013-01-01 00:00:00,NaN,4.244,26.546,48.246,NaN,110.9530,NaN
2013-01-01 01:00:00,NaN,1.000,9.654,56.526,NaN,119.6280,NaN
2013-01-01 02:00:00,NaN,1.000,8.700,54.750,NaN,15.0530,NaN
2013-01-01 03:00:00,NaN,1.000,9.163,53.035,NaN,10.9130,NaN
2013-01-01 04:00:00,NaN,1.000,8.430,52.995,NaN,11.4570,NaN
2013-01-01 05:00:00,NaN,1.000,9.163,51.339,NaN,12.2130,NaN
2013-01-01 06:00:00,NaN,1.000,8.790,51.917,NaN,10.6060,NaN
2013-01-01 07:00:00,NaN,1.000,8.635,54.391,NaN,9.2950,NaN
2013-01-01 08:00:00,NaN,1.000,8.744,56.885,NaN,9.6530,NaN


In [27]:
for column in df.columns:
    print(column)

AirQualityStationEoICode
UnitOfMeasurement
As in PM10
BaP in PM10
Benzo(a)anthracene in PM10
Benzo(b)fluoranthene in PM10
Benzo(j)fluoranthene in PM10
Benzo(k)fluoranthene in PM10
C6H5-CH3
C6H6
Cd in PM10
Dibenzo(ah)anthracene in PM10
EC in PM10
Indeno-(1,2,3-cd)pyrene in PM
NO
NOX as NO2
Ni in PM10
O3
OC in PM10
PM10
PM2.5
Pb in PM10
SO2


In [28]:
len(df.columns)

23

In [69]:
df2 = pd.concat([df[['C6H6']], df[['O3']]])
df3 = pd.concat([df2, df[['O3']]])

In [87]:
df3 = pd.DataFrame([[np.nan, 3., 5., 1], [-4.6, np.nan, np.nan, 2], [np.nan, 7., np.nan, 3.]])
df4 = pd.DataFrame([[-42.6, np.nan, -8.2], [-5., 1.6, 4]], index=[1, 2])
result = df4.combine_first(df3)
result2 = df3.combine_first(df4)

In [33]:
so2 = df['SO2'].dropna()
so2 = so2[~so2.index.duplicated(keep='first')]
print(so2.index[0], '-', so2.index[-1], ' length = ', len(so2))

2013-01-01 00:00:00 - 2013-12-31 23:00:00  length =  8371


In [34]:
so2 = so2.drop(pd.Timestamp('2013-01-01 04:00:00'))
len(so2)

8370

In [35]:
so2 = so2.resample('H').interpolate(method='time')

In [36]:
so2.head(7)

Timestamp
2013-01-01 00:00:00    27.4580
2013-01-01 01:00:00    10.5140
2013-01-01 02:00:00     5.5610
2013-01-01 03:00:00     4.9480
2013-01-01 04:00:00     4.5115
2013-01-01 05:00:00     4.0750
2013-01-01 06:00:00     3.7150
Freq: H, Name: SO2, dtype: float64

In [37]:
len(so2)

8760